## SimulatedLossを試す

In [ ]:
from pathlib import Path

import numpy as np
import tensorflow as tf

import stock

In [ ]:
dataset_params = stock.dl.dataset.DatasetParams(
    symbols_csv_path=Path("~/work/Projects/stock/data/sp500_companies.csv"),
    data_dir=stock.DATA_DIR / "sp500",
    dataset_path=stock.DATA_DIR / "dataset" / "sp500_20221203.npy",
    input_width=30,
    shift=30,
    output_width=1,
)
model_params = stock.dl.models.ModelParams(name="lstm")
loss_params = stock.dl.losses.SimulatedLossParams(name="simulated")
train_params = stock.dl.train.TrainerParams(
    dataset_params=dataset_params, 
    model_params=model_params, 
    loss_params=loss_params,
    epochs=10
)

In [ ]:
trainer = stock.dl.train.Trainer(train_params)

trainer.params.n_output_classes = trainer.dataset.num_symbols * 3
trainer.params.loss_params.n_classes = trainer.dataset.num_symbols
trainer.params.loss_params.high_lows = trainer.dataset.high_low_indices

trainer.build()

In [ ]:
trainer.model.summary()

In [ ]:
params = stock.dl.losses.SimulatedLossParams(
    name="foo",
    n_classes=3,
    high_lows=[(0, 1), (2, 3), (4, 5)]
)
sl = stock.dl.losses.SimulatedLoss(params)

In [ ]:
y_true = np.array([
    # [200, 100, 250, 190, 280, 250],
    [10, 5, 7, 6, 5, 5],
    # [21, 11, 22, 15, 23, 8],
    # [150, 10, 140, 30, 20, 10],
], dtype=np.float32)
y_pred = np.array([
    # [0.1, 0.2, 0.7, 190, 160, 200, 195, 290, 240],
    [0.3, 0.3, 0.4, 10, 6, 7, 7, 4, 6],
    # [0.2, 0.2, 0.6, 20, 18, 19, 19, 22, 7],
    # [0.0, 0.2, 0.8, 150, 10, 140, 30, 20, 10],
], dtype=np.float32)

In [ ]:
n_classes = 3
fee_rate = 0.005
idx = 2
high = y_true[:, 4]
low = y_true[:, 5]
# high = [3.0]
# low = [11.0]
profit_loss_weight: float = 1.0
variance_loss_weight: float = 1.0
no_contract_weight: float = 0.1  # 何も取引しなかった場合のペナルティの重み

ratio = y_pred[:, idx]  # 資産の何割をこの銘柄の売買に使うか
sell = y_pred[:, n_classes + idx] * (1.0 - fee_rate)  # 買値
buy = y_pred[:, n_classes * 2 + idx] * (1.0 + fee_rate)  # 売値

# 儲け額
profit = ratio * tf.abs(sell - buy) / (tf.abs(sell + buy)  + 1e-5)
# 高値、安値の両方で取引成立しなかった場合は0
# 高値、安値のどちらかのみで取引成立した場合は、マイナス
# 高値、安値の両方で取引成立した場合は、プラス
factor = (high - sell) * sl.relu(buy - low) + sl.relu(high - sell) * (buy - low)
# 高値、安値の両方で取引成立しなかった場合はマイナスfactorをかける
factor -= no_contract_weight * sl.relu(sell - high) * sl.relu(low - buy)
# 銘柄ごとの価格差によるscaleを考慮して補正
factor /= tf.square((high + low + 1e-3) / 100.0)
total_profit = tf.reduce_sum(profit * factor)


In [ ]:
-total_profit

In [ ]:
print(f"high = {high}, low = {low}, sell = {sell}, buy = {buy})")
print(f"ratio = {ratio}, profit = {profit}, factor = {factor}, total_profit = {total_profit}")
print(f"{profit * factor}")

In [ ]:
sl(y_true, y_pred)

sl.profit_loss, sl.variance_loss